# Prospecção de Dados (Data Mining) DI/FCUL - Project

## Project (MC/DI/FCUL - 2024)

### GROUP:`09`

* Afonso Gama, 55857 - x Hours
* Eduardo Carneiro, 62515 - 2 Hours
* Guilherme Rosario, 62543 - x Hours
* Marco Viana, 62550 - x Hours

# TODO list:


In [1]:
import pickle

In [4]:
# Load the dataset
with open('../data/mol_bits.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
print(type(data))
print(data)

---

In [22]:
# Load the data from the csvs
import pandas as pd
import numpy as np

df_train = pd.read_csv("../data/activity_train.csv", header=None)
df_test = pd.read_csv("../data/activity_test_blanked.csv", header=None)

In [30]:
# Pre-Processing
from sklearn.preprocessing import LabelEncoder

# Train
# Get the input arrays from the df
X_train_labels_x1 = df_train.iloc[:,0].to_numpy()
X_train_labels_x2 = df_train.iloc[:,1].to_numpy()

# Get unique values in x1 and convert them into integers
label_enc_x1 = LabelEncoder()
label_enc_x1.fit(X_train_labels_x1)
X_train_transformed_x1 = label_enc_x1.transform(X_train_labels_x1)

# Get unique values in x2 and convert them into integers
label_enc_x2 = LabelEncoder()
label_enc_x2.fit(X_train_labels_x2)
X_train_transformed_x2 = label_enc_x2.transform(X_train_labels_x2)

# ---

# Test
# Get the input arrays from the df
X_test_labels_x1 = df_test.iloc[:,0].to_numpy()
X_test_labels_x2 = df_test.iloc[:,1].to_numpy()

# Convert values into integers
X_test_transformed_x1 = label_enc_x1.transform(X_test_labels_x1)

# Get unique values in x2 and convert them into integers
label_enc_x2_test = LabelEncoder()
label_enc_x2_test.fit(X_test_labels_x2)
X_test_transformed_x2 = label_enc_x2_test.transform(X_test_labels_x2)

In [31]:
# Get final inputs for the model
X_train = np.column_stack((X_train_transformed_x1, X_train_transformed_x2))
Y_train = df_train.iloc[:,2].to_numpy()

X_test = np.column_stack((X_test_transformed_x1, X_test_transformed_x2))

In [32]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import make_regression

# SVM
# NN ??
# print(X)

# Train the model
model_rfr = RandomForestRegressor(n_estimators=100, random_state=42)
model_rfr.fit(X_train, Y_train)

# Get the predictions
rfr_predictions = model_rfr.predict(X_test)

array([4.93, 5.17, 5.17, ..., 2.1 , 2.1 , 2.1 ])